### Install dependencies

In [1]:
%env HDF5_USE_FILE_LOCKING=FALSE
%env WANDB_MODE=dryrun
%env WANDB_NOTEBOOK_NAME=deepmr

env: HDF5_USE_FILE_LOCKING=FALSE
env: WANDB_MODE=dryrun
env: WANDB_NOTEBOOK_NAME=deepmr


In [2]:
import bpnet
from bpnet.cli.contrib import ContribFile
from bpnet.plot.tracks import plot_tracks, to_neg

import os
import uuid
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output, HTML
from pathlib import Path
import pandas as pd
import numpy as np
clear_output()

#### Optional: Setup wandb

In [3]:
import wandb

wandb.init(project='deepmr', entity='an1lam', mode='disabled')

In [4]:
# config variables
n_reps = 5

# file paths
config_dir = Path('./bpnet/') 

model_config_fname = 'ChIP-nexus-default.gin'
data_config_fname = 'ChIP-nexus.dataspec.yml'

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
project_dir = '/home/ubuntu/dev/an1lam/deepmr/'
data_dir = os.path.join(project_dir, 'dat')
output_dir = os.path.join(data_dir, f'res-bpnet-training-{timestamp}')
output_dir

'/home/ubuntu/dev/an1lam/deepmr/dat/res-bpnet-training-2021-01-02-20-11-07'

In [5]:
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'output_ensemble'), exist_ok=True)

In [6]:
!cat {config_dir}/{data_config_fname}

task_specs:
  Oct4:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/idr-optimal-set.summit.bed.gz
  Sox2:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz
  Nanog:
    tracks:
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.pos.bw
    - /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.neg.bw
    peaks: /home/ubuntu/dev/an1lam/deepmr/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/idr-optim

### Data stats

In [7]:
# chromsomome names of differnet peaks
!zcat {project_dir}dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz \
    | cut -f 1 | sort -u

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX
chrY


Each task (or TF) can specify a set of peaks associated with it. Here are the number of peaks per TF we will use in this tutorial:

In [8]:
tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']

# number of peaks per task
for task in tasks:
    print(task)
    !zcat {data_dir}/bpnet-manuscript-data/data/chip-nexus/{task}/idr-optimal-set.summit.bed.gz | wc -l

Oct4
25849
Sox2
10999
Nanog
56459
Klf4
57601


## 2. Train the model

Having specified `dataspec.yml`, we are now ready to train the model with 

```
bpnet train <dataspec.yml> <output dir> [optional flags]`
```


We will use a pre-made model [bpnet9](../bpnet/premade/bpnet9.gin) as a starting point and modify a few parameters specified in the config.gin file. Specifically, we will 
- train the model only on chromosomes 16-19
- evaluate the model on chromosome 2
- use only 3 layers of dilated convolutions 
- use an input sequence length of 200 bp and accordingly lower the augmentation shift to 100 bp

In [9]:
!cat {config_dir}/{model_config_fname} 
# NOTE: test_chr will be also excluded similar to 'exclude_chr'

b_loss_weight = 0
c_loss_weight = 10
p_loss_weight = 1
filters = 64
tconv_kernel_size = 25
lr = 0.004
n_dil_layers = 9
train.batch_size = 128
merge_profile_reg = False
dataspec = 'ChIP-nexus.dataspec.yml'

batchnorm = False

padding = 'same'
seq_width = 1000

tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']


Have a look at the original gin file of bpnet9 here: https://github.com/kundajelab/bpnet/blob/master/bpnet/premade/bpnet9-ginspec.gin. For more information on using gin files see <https://github.com/google/gin-config>. 

To track model training and evaluation, we will use [wandb](http://wandb.com/) by adding `--wandb=avsec/bpnet-demo` to `bpnet train`. You can navigate to https://app.wandb.ai/avsec/bpnet-demo to see the training progress.

Let's train!

In [10]:
# setup all the file paths
example_model_dir = os.path.join(output_dir, 'output_ensemble', '0')

In [ ]:
# Train for at most 10 epochs
for i in range(n_reps):
    # setup a new run_id (could be done automatically, but then the output directory would change)
    run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "_" + str(uuid.uuid4())
    !cd {config_dir} && bpnet train {data_config_fname} --premade=bpnet9 \
        --config={model_config_fname} {output_dir} \
        --run-id '{run_id}' \
        --override='train.epochs=10; train.seed={i}'
    # softlink the new output directory
    !rm -rf {output_dir}/output_ensemble/{i} && ln -srf {output_dir}/{run_id} {output_dir}/output_ensemble/{i}

Using TensorFlow backend.

2021-01-02 20:12:25,657 [WARNING] From /home/ubuntu/anaconda3/envs/deepmr/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2021-01-02 20:12:26,266 [INFO] NumExpr defaulting to 4 threads.
/home/ubuntu/anaconda3/envs/deepmr/lib/python3.6/site-packages/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
INFO [01-02 20:12:27] Using gpu: 0, memory fraction: 0.45
2021-01-02 20:12:27.988359: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-01-02 20:12:27.996895: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300065000 Hz
2021-01-02

Epoch 1/10
2021-01-02 20:12:45.388201: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2021-01-02 20:12:45.570487: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
710/710 [==============================] - 257s 362ms/step - loss: 3498.4180 - Oct4/profile_loss: 894.8764 - Oct4/counts_loss: 0.6223 - Sox2/profile_loss: 466.8994 - Sox2/counts_loss: 0.9620 - Nanog/profile_loss: 1153.9167 - Nanog/counts_loss: 1.9513 - Klf4/profile_loss: 938.4981 - Klf4/counts_loss: 0.8871 - val_loss: 3348.5237 - val_Oct4/profile_loss: 830.0644 - val_Oct4/counts_loss: 0.4463 - val_Sox2/profile_loss: 464.7971 - val_Sox2/counts_loss: 0.5202 - val_Nanog/profile_loss: 1107.8526 - val_Nanog/counts_loss: 1.2726 - val_Klf4/profile_loss: 916.9908 - val_Klf4/counts_loss: 0.6429
Epoch 2/10
710/710 [==============================] - 252s 355ms/step - loss: 3275.4454 - Oct4/profile_loss:

Executing:  10%|███▏                           | 3/29 [00:04<00:39,  1.51s/cell]2021-01-02 21:07:55.141552: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-01-02 21:07:55.151398: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300065000 Hz
2021-01-02 21:07:55.151694: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563c104c9e30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-01-02 21:07:55.151732: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-01-02 21:07:55.154670: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-01-02 21:07:55.224128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be 

OrderedDict([('valid-peaks', {'Oct4/profile/binsize=1/auprc': 0.18009270799318408, 'Oct4/profile/binsize=1/random_auprc': 0.002962928723389025, 'Oct4/profile/binsize=1/n_positives': 49407, 'Oct4/profile/binsize=1/frac_ambigous': 0.07197004608294931, 'Oct4/profile/binsize=1/imbalance': 0.0029207034942389304, 'Oct4/profile/binsize=10/auprc': 0.4895772090647788, 'Oct4/profile/binsize=10/random_auprc': 0.03456102474822499, 'Oct4/profile/binsize=10/n_positives': 39490, 'Oct4/profile/binsize=10/frac_ambigous': 0.36034507351327627, 'Oct4/profile/binsize=10/imbalance': 0.03386899927356185, 'Oct4/counts/mse': 0.28948304, 'Oct4/counts/var_explained': 0.3258916735649109, 'Oct4/counts/pearsonr': 0.5844209726909169, 'Oct4/counts/spearmanr': 0.5507967253432144, 'Oct4/counts/mad': 0.40735608, 'Sox2/profile/binsize=1/auprc': 0.3745101241906123, 'Sox2/profile/binsize=1/random_auprc': 0.005811615354319482, 'Sox2/profile/binsize=1/n_positives': 17617, 'Sox2/profile/binsize=1/frac_ambigous': 0.07234169764

INFO [01-02 21:10:05] Using gpu: 0, memory fraction: 0.45
2021-01-02 21:10:05.740941: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-01-02 21:10:05.749769: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300065000 Hz
2021-01-02 21:10:05.750058: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560f2b9b5dd0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-01-02 21:10:05.750094: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-01-02 21:10:05.753122: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-01-02 21:10:05.823465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node

/home/ubuntu/anaconda3/envs/deepmr/lib/python3.6/site-packages/bpnet/cli/train.py:208: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  .replace(" = ", ": "))
Epoch 1/10
2021-01-02 21:10:23.493162: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2021-01-02 21:10:23.678954: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
710/710 [==============================] - 267s 377ms/step - loss: 3509.9893 - Oct4/profile_loss: 888.0290 - Oct4/counts_loss: 0.5341 - Sox2/profile_loss: 473.3264 - Sox2/counts_loss: 0.6434 - Nanog/profile_loss: 1171.8839 - Nanog/counts_loss: 1.0019 - Klf4/profile_loss: 945.3604 - Klf4/counts_loss: 0.9595 - val_loss: 3378.8544 - val_Oct4/profile_loss: 838.9644 - val_Oct4/counts_loss: 0.4144 - val_Sox2/profile_loss: 466.346

In [ ]:
! echo {output_dir}
! echo
! ls {output_dir}/output_ensemble/0

In [ ]:
!ls -latr {example_model_dir}/